In [8]:
# Import the necessary modules for the protocol
import ee as ee
ee.Initialize()
import pandas as pd
from scipy.spatial import ConvexHull
from sklearn.decomposition import PCA
import numpy as np
from itertools import combinations

Traceback (most recent call last):
  File "/Users/johanvandenhoogen/opt/anaconda3/envs/ee/lib/python3.6/site-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/johanvandenhoogen/opt/anaconda3/envs/ee/lib/python3.6/site-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/johanvandenhoogen/opt/anaconda3/envs/ee/lib/python3.6/site-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoun

In [9]:
# Input the proportion of variance that you would like to cover when running the script
propOfVariance = 90



In [10]:
def assessExtrapolation(importedData, compositeImage, propOfVariance):
    
    # Excise the columns of interest from the data frame
    variablesOfInterest = importedData
#     variablesOfInterest = importedData.drop(['system:index', '.geo'], axis=1)
    
    # Compute the mean and standard deviation of each band, then standardize the point data
    meanVector = variablesOfInterest.mean()
    stdVector = variablesOfInterest.std()
    standardizedData = (variablesOfInterest-meanVector)/stdVector
    
    # Then standardize the composite from which the points were sampled
    meanList = meanVector.tolist()
    stdList = stdVector.tolist()
    bandNames = list(meanVector.index)
    meanImage = ee.Image(meanList).rename(bandNames)
    stdImage = ee.Image(stdList).rename(bandNames)
    standardizedImage = compositeImage.subtract(meanImage).divide(stdImage)
    
    # Run a PCA on the point samples
    pcaOutput = PCA()
    pcaOutput.fit(standardizedData)
    
    # Save the cumulative variance represented by each PC
    cumulativeVariance = np.cumsum(np.round(pcaOutput.explained_variance_ratio_, decimals=4)*100)
    
    # Make a list of PC names for future organizational purposes
    pcNames = ['PC'+str(x) for x in range(1,variablesOfInterest.shape[1]+1)]
    
    # Get the PC loadings as a data frame
    loadingsDF = pd.DataFrame(pcaOutput.components_,columns=[str(x)+'_Loads' for x in bandNames],index=pcNames)
    
    # Get the original data transformed into PC space
    transformedData = pd.DataFrame(pcaOutput.fit_transform(standardizedData,standardizedData),columns=pcNames)
    
    # Make principal components images, multiplying the standardized image by each of the eigenvectors
    # Collect each one of the images in a single image collection;
    
    # First step: make an image collection wherein each image is a PC loadings image
    listOfLoadings = ee.List(loadingsDF.values.tolist());
    eePCNames = ee.List(pcNames)
    zippedList = eePCNames.zip(listOfLoadings)
    def makeLoadingsImage(zippedValue):
        return ee.Image.constant(ee.List(zippedValue).get(1)).rename(bandNames).set('PC',ee.List(zippedValue).get(0))
    loadingsImageCollection = ee.ImageCollection(zippedList.map(makeLoadingsImage))
    
    # Second step: multiply each of the loadings image by the standardized image and reduce it using a "sum"
    # to finalize the matrix multiplication
    def finalizePCImages(loadingsImage):
        return ee.Image(loadingsImage).multiply(standardizedImage).reduce('sum').rename([ee.String(ee.Image(loadingsImage).get('PC'))]).set('PC',ee.String(ee.Image(loadingsImage).get('PC')))
    principalComponentsImages = loadingsImageCollection.map(finalizePCImages)
    
    # Choose how many principal components are of interest in this analysis based on amount of
    # variance explained
    numberOfComponents = sum(i < propOfVariance for i in cumulativeVariance)+1
    print('Number of Principal Components being used:',numberOfComponents)
    
    # Compute the combinations of the principal components being used to compute the 2-D convex hulls
    tupleCombinations = list(combinations(list(pcNames[0:numberOfComponents]),2))
    print('Number of Combinations being used:',len(tupleCombinations))
    
    # Generate convex hulls for an example of the principal components of interest
    cHullCoordsList = list()
    for c in tupleCombinations:
        firstPC = c[0]
        secondPC = c[1]
        outputCHull = ConvexHull(transformedData[[firstPC,secondPC]])
        listOfCoordinates = transformedData.loc[outputCHull.vertices][[firstPC,secondPC]].values.tolist()
        flattenedList = [val for sublist in listOfCoordinates for val in sublist]
        cHullCoordsList.append(flattenedList)
    
    # Reformat the image collection to an image with band names that can be selected programmatically
    pcImage = principalComponentsImages.toBands().rename(pcNames)
    
    # Generate an image collection with each PC selected with it's matching PC
    listOfPCs = ee.List(tupleCombinations)
    listOfCHullCoords = ee.List(cHullCoordsList)
    zippedListPCsAndCHulls = listOfPCs.zip(listOfCHullCoords)
    
    def makeToClassifyImages(zippedListPCsAndCHulls):
        imageToClassify = pcImage.select(ee.List(zippedListPCsAndCHulls).get(0)).set('CHullCoords',ee.List(zippedListPCsAndCHulls).get(1))
        classifiedImage = imageToClassify.rename('u','v').classify(ee.Classifier.spectralRegion([imageToClassify.get('CHullCoords')]))
        return classifiedImage
    classifedImages = ee.ImageCollection(zippedListPCsAndCHulls.map(makeToClassifyImages))
    finalImageToExport = classifedImages.sum().divide(ee.Image.constant(len(tupleCombinations)))
    
    return finalImageToExport



In [11]:
depthList = [[0,5],[5,15]]
monthList = list(range(1,13))

# Load a geometry to use for the export
exportingGeometry = ee.Geometry.Polygon([[[-180, 88], [180, 88], [180, -88], [-180, -88]]], None, False);

In [12]:
for depth in depthList:
    print('Depth interval:', depth[0], '-', depth[1], 'cm')
    PCAimageList = []
    for month in monthList:  
        print(month)
        
        # Input a list of the covariates being used
        staticCovarList = [
        'Aridity_Index',
        'EarthEnvTopoMed_1stOrderPartialDerivEW',
        'EarthEnvTopoMed_Elevation',
        'EarthEnvTopoMed_Roughness',
        'EarthEnvTopoMed_TerrainRuggednessIndex',
        'GlobBiomass_AboveGroundBiomass',
        'Human_Development_Percentage',
        'LandCoverClass_Barren',
        'Nadir_Reflectance_Band1',
        'Nadir_Reflectance_Band2',
        'Nadir_Reflectance_Band3',
        'Nadir_Reflectance_Band4',
        'Nadir_Reflectance_Band5',
        'Nadir_Reflectance_Band6',
        'Nadir_Reflectance_Band7',
        'NDVI',
        'PET',
        'Population_Density',
        'SG_Absolute_depth_to_bedrock',
        'SG_Bulk_density_005cm',
        'SG_Depth_to_bedrock',
        'SG_Sand_Content_005cm',
        'SG_SOC_Density_005cm',
        'SG_Soil_pH_H2O_005cm'
        ];

        # Monthly variable names
        monthly_vars = [
        "EarthEnvCloudCover_MODCF_monthlymean_"+str(month).zfill(2),
        "WorldClim2_H2OVaporPressure_Month"+str(month).zfill(2),
        "WorldClim2_SolarRadiation_Month"+str(month).zfill(2)
        ];

        # Cloud cover has NA values for December
        if month == 1 or month == 12:
            del monthly_vars[0]
        else:
            pass
        
        classProperty = 'deltaT_'+str(month).zfill(2)

        # Load the composite on which to perform the mapping, and subselect the bands of interest
        compositeToUse = ee.Image("projects/crowtherlab/johan/CrowtherLab_Composite_30ArcSec_2019").select(staticCovarList+monthly_vars)

        # Monthly climate variables from CHELSA
        CHELSA_monthly = ee.Image('users/johanvandenhoogen/000_CHELSA_monthly/CHELSA_monthly_'+str(month).zfill(2))

        # CHELSA variable names
        CHELSA_vars = CHELSA_monthly.bandNames().getInfo()
    
        # Monthly snow cover
        snowCover = ee.Image('users/johanvandenhoogen/000_composites/monthly_snowcover/SnowCover_'+str(month).zfill(2)).rename("snowCover_"+str(month).zfill(2))

        # Snowcover variable name
        snowCover_var = "snowCover_"+str(month).zfill(2)
    
        # Full list of variables to be included
        covariateList = staticCovarList+monthly_vars+CHELSA_vars+[snowCover_var]
        
        # Construct final composite
        compositeToClassify = ee.Image.cat(
            compositeToUse,
            CHELSA_monthly,
            snowCover
            )

        # Import data
        sampled_data = pd.read_csv('data/training_data/'+str(depth[0])+'_'+str(depth[1])+'cm_/'+classProperty+'/'+classProperty+'CV_Fold_Collection.csv')[covariateList]
                
        # Apply the function
        finalImageToExport_SoilT = assessExtrapolation(sampled_data, compositeToClassify, propOfVariance)
        
        PCAimageList.append(finalImageToExport_SoilT.toFloat())
        
        task = ee.batch.Export.image.toAsset(
            image = finalImageToExport_SoilT.toFloat(),
            description = str(depth[0])+'_'+str(depth[1])+classProperty+'PCA_CHull_IntExt',
            assetId = 'users/johanvandenhoogen/2020_soil_temperature_v6/PCA_int_extImgs/'+str(depth[0])+'_'+str(depth[1])+'_'+classProperty+'_PCA_CHull_IntExt',
            crs = 'EPSG:4326',
            crsTransform = '[0.008333333333333333,0,-180,0,-0.008333333333333333,90]',
            region = exportingGeometry,
            maxPixels = int(1e13)
        );

#         task.start()
        
    meanImage = ee.ImageCollection.fromImages(PCAimageList).reduce(
        reducer = ee.Reducer.mean()
    )

    meanImageExport = ee.batch.Export.image.toAsset(
        image = meanImage.toFloat(),
        description = str(depth[0])+'_'+str(depth[1])+'_PCA_int_ext_mean_image',
        assetId = 'users/johanvandenhoogen/2020_soil_temperature_v6/'+str(depth[0])+'_'+str(depth[1])+'_PCA_int_ext_mean_image' ,
        crs = 'EPSG:4326',
        crsTransform = '[0.008333333333333333,0,-180,0,-0.008333333333333333,90]',
        region = exportingGeometry,
        maxPixels = int(1e13)
    );
    meanImageExport.start()

    meanImageExport = ee.batch.Export.image.toDrive(
        image = meanImage.multiply(100).toInt().toFloat().divide(100),
        description = str(depth[0])+'_'+str(depth[1])+'_PCA_int_ext_mean_image',
        folder = 'soil_temperature',
        crs = 'EPSG:4326',
        crsTransform = '[0.04166666666666667,0,-180,0,-0.04166666666666667,90]',
        region = exportingGeometry,
        maxPixels = int(1e13)
    );

#     meanImageExport.start()



Depth interval: 0 - 5 cm
1
Number of Principal Components being used: 11
Number of Combinations being used: 55
2
Number of Principal Components being used: 11
Number of Combinations being used: 55
3
Number of Principal Components being used: 11
Number of Combinations being used: 55
4
Number of Principal Components being used: 11
Number of Combinations being used: 55
5
Number of Principal Components being used: 12
Number of Combinations being used: 66
6
Number of Principal Components being used: 12
Number of Combinations being used: 66
7
Number of Principal Components being used: 12
Number of Combinations being used: 66
8
Number of Principal Components being used: 12
Number of Combinations being used: 66
9
Number of Principal Components being used: 12
Number of Combinations being used: 66
10
Number of Principal Components being used: 11
Number of Combinations being used: 55
11
Number of Principal Components being used: 12
Number of Combinations being used: 66
12
Number of Principal Comp